In [ ]:
# ENIGMA-DTI-TBSS-Protocol

#Set Up
 cp /subject*_folder/subject*_FA.nii.gz /enigmaDTI/TBSS/run_tbss/

cd /enigmaDTI/TBSS/run_tbss/

# TBSS
 tbss_1_preproc *.nii.gz
tbss_2_reg -t ENIGMA_DTI_FA.nii.gz
 tbss_3_postreg -S

mkdir /enigmaDTI/TBSS/run_tbss/BAD_REGISTER/
mv FA_didnt_pass_QC* /enigmaDTI/TBSS/run_tbss/BAD_REGISTER/

mkdir /enigmaDTI/TBSS/ENIGMA_targets_edited/

# Create Common Maks for Cohort 
cd /enigmaDTI/TBSS/run_tbss/

${FSLPATH}/fslmerge –t ./all_FA_QC ./FA/*FA_to_target.nii.gz
${FSLPATH}/fslmaths ./all_FA_QC -bin -Tmean –thr 0.9 /enigmaDTI/TBSS/ENIGMA_targets_edited/mean_FA_mask.nii.gz

${FSLPATH}/fslmaths /enigmaDTI/TBSS/ENIGMA_targets/ENIGMA_DTI_FA.nii.gz –mas /enigmaDTI/TBSS/ENIGMA_targets_edited/mean_FA_mask.nii.gz /enigmaDTI/TBSS/ENIGMA_targets_edited/mean_FA.nii.gz

${FSLPATH}/fslmaths /enigmaDTI/TBSS/ENIGMA_targets/ENIGMA_DTI_FA_skeleton.nii.gz –mas /enigmaDTI/TBSS/ENIGMA_targets_edited/mean_FA_mask.nii.gz /enigmaDTI/TBSS/ENIGMA_targets_edited/mean_FA_skeleton.nii.gz

tbss_4_prestats -0.049

# Parallelisation 
cd /enigmaDTI/TBSS/run_tbss/

 for subj in subj_1 subj_2 … subj_N
 do

 mkdir -p ./FA_individ/${subj}/stats/
 mkdir -p ./FA_individ/${subj}/FA/

 cp ./FA/${subj}_*.nii.gz ./FA_individ/${subj}/FA/
 ${FSLPATH}/fslmaths ./FA_individ/${subj}/FA/${subj}_*FA_to_target.nii.gz -mas /enigmaDTI/TBSS/ENIGMA_targets_edited/mean_FA_mask.nii.gz ./FA_individ/${subj}/FA/${subj}_masked_FA.nii.gz

 done

# Skeletonise images by projecting the ENIMGA skeleton
cd /enigmaDTI/TBSS/run_tbss/
 for subj in subj_1 subj_2 … subj_N
 do

 ${FSLPATH}/tbss_skeleton -i ./FA_individ/${subj}/FA/${subj}_masked_FA.nii.gz -p 0.049 /enigmaDTI/TBSS/ENIGMA_targets_edited/mean_FA_skeleton_mask_dst ${FSLPATH}/data/standard/LowerCingulum_1mm.nii.gz ./FA_individ/${subj}/FA/${subj}_masked_FA.nii.gz ./FA_individ/${subj}/stats/${subj}_masked_FAskel.nii.gz -s /enigmaDTI/TBSS/ENIGMA_targets_edited/mean_FA_skeleton_mask.nii.gz

 done

In [ ]:
# Protocol for ROI analysis using the ENIGMA-DTI template
# run_ENIGMA_ROI_ALL_script.sh
#!/bin/bash
#$ -S /bin/bash


## neda.jahanshad@ini.usc.edu ##
## ENIGMA-DTI ##

#######
## part 1 - loop through all subjects to create a subject ROI file 
#######

#make an output directory for all files
mkdir ENIGMA_ROI_part1
dirO1=./ENIGMA_ROI_part1/

for subject in Subject1 Subject7

do
./singleSubjROI_exe ENIGMA_look_up_table.txt mean_FA_skeleton.nii.gz JHU-WhiteMatter-labels-1mm.nii.gz ${dirO1}/${subject}_ROIout ${subject}_masked_FAskel.nii.gz

done


#######
## part 2 - loop through all subjects to create ROI file 
##			removing ROIs not of interest and averaging others
#######

#make an output directory for all files
mkdir ENIGMA_ROI_part2
dirO2=./ENIGMA_ROI_part2/

# you may want to automatically create a subjectList file 
#    in which case delete the old one
#    and 'echo' the output files into a new name
rm ./subjectList.csv

for subject in Subject1 Subject7

do
./averageSubjectTracts_exe ${dirO1}/${subject}_ROIout.csv ${dirO2}/${subject}_ROIout_avg.csv


# can create subject list here for part 3!
echo ${subject},${dirO2}/${subject}_ROIout_avg.csv >> ./subjectList.csv
done


#######
## part 3 - combine all 
#######
Table=./ALL_Subject_Info.txt
subjectIDcol=subjectID
subjectList=./subjectList.csv
outTable=./combinedROItable.csv
Ncov=2
covariates="Age;Sex"
Nroi="all" #2
rois="IC;EC"

#location of R binary 
Rbin=/usr/local/R-2.9.2_64bit/bin/R

#Run the R code
${Rbin} --no-save --slave --args ${Table} ${subjectIDcol} ${subjectList} ${outTable} ${Ncov} ${covariates} ${Nroi} ${rois} <  ./combine_subject_tables.R  



# run_combineSubjectROI_script.sh
#!/bin/bash
#$ -S /bin/bash


Table=./ALL_Subject_Info.txt
subjectIDcol=subjectID
subjectList=./subjectList.csv
outTable=./combinedROItable.csv
Ncov=2
covariates="Age;Sex"
Nroi="all" #2
rois="IC;EC"

#R binary
Rbin=/usr/local/R-2.9.2_64bit/bin/R

#Run the R code
${Rbin} --no-save --slave --args ${Table} ${subjectIDcol} ${subjectList} ${outTable} ${Ncov} ${covariates} ${Nroi} ${rois} <  ./combine_subject_tables.R  

# Combine_subject_tables.R
#/*
# * combine_subject_tables.R
# * 
# * Created by Neda Jahanshad - neda.jahanshad@ini.usc.edu
# * ENIGMA_DTI
# */

#################
cmdargs = commandArgs(trailingOnly=T);
TableFile=cmdargs[1]
subjectIDcol=cmdargs[2]
subjectList	=cmdargs[3]
outTable=cmdargs[4]
Ncov=as.numeric(cmdargs[5]);
covariates=cmdargs[6]
Nroi=as.numeric(cmdargs[7]);
NroiC=cmdargs[7];
rois=cmdargs[8]
#################
#Table<-"./ALL_INFO.txt"
#subjectIDcol="subjectID"
#subjectList="./subjectList.csv"
#outTable="./FULL_ROItable.csv"
#Ncov=2
#ovariates="Age;Sex"
#roi=2
#NroiC="2"
#rois="IC;EC"
################
if (substring(TableFile,nchar(TableFile)-3,nchar(TableFile))==".csv"){
	tokT=",";
} else {tokT="\t"} 
if (substring(subjectList,nchar(subjectList)-3,nchar(subjectList))==".csv"){
	tokS=",";
} else {tokS="\t"}

###################
##filter out table to include only variables of interest
###################
Table<-read.table(TableFile,sep=tokT,header=T,blank.lines.skip = TRUE)
columnnames<-colnames(Table)
#print(columnnames)
#print(subjectIDcol)
DesignMatrix<-data.frame(subjectID=Table[,which(columnnames==subjectIDcol)])
#print(DesignMatrix$subjectID)

if (Ncov > 0) {
	parsedCov=parse(text=covariates)
	for (nc in 1:Ncov) {
		covName<-as.character(parsedCov[nc])
		print(covName)
		origcolnames = colnames(DesignMatrix);
		DesignMatrix[,length(DesignMatrix)+1] = Table[,which(columnnames==covName)]
		colnames(DesignMatrix)<-c(origcolnames,covName)
	}
}

iNA=which(apply(DesignMatrix,1,function(x)any(is.na(x))));
if (length(iNA > 0)) {
	DesignMatrix<-DesignMatrix[-which(apply(DesignMatrix,1,function(x)any(is.na(x)))),]
}


##############
### match subjects in subjectList
##############

### match file for subjects 
subjList<-data.frame(read.table(subjectList,sep=tokS,header=FALSE,blank.lines.skip = TRUE))
files=subjList[,2]
matchind = match(DesignMatrix$subjectID,subjList[,1]);
#print(as.character(subjList[,1]))

### for each subject ID, read file and add columns corresponding to ROIs
done=0;
for (i in 1:length(as.vector(matchind)) ) {
	f_indx = matchind[i]
	### read file
	Rfile=as.character(files[f_indx]);

	if (file.exists(Rfile)==1) {
		if(done==0) {
		if (NroiC=="all") {
			ROImatrix<-data.matrix(read.table(as.character(Rfile),sep=",",header=T,blank.lines.skip = TRUE,na.strings = "NaN",row.names=1))
			rownames=row.names(ROImatrix)
			for (r in 1:length(rownames)) {
			origcolnames = colnames(DesignMatrix);
			DesignMatrix[,length(DesignMatrix)+1] = rep(NA,length(as.vector(matchind)))
			colnames(DesignMatrix)<-c(origcolnames,rownames[r])
			}
#DesignMatrix$Tract=NULL
		} else if (Nroi > 0) {
			parsedROIs=parse(text=rois)
			for (nc in 1:Nroi) {
				roiName<-as.character(parsedROIs[nc])
				print(roiName)
				origcolnames = colnames(DesignMatrix);
				DesignMatrix[,length(DesignMatrix)+1] = rep(NA,length(as.vector(matchind)))
				colnames(DesignMatrix)<-c(origcolnames,roiName)
			}
		}
		done=1;
	}
	
	ROImatrix<-data.matrix(read.table(as.character(Rfile),sep=",",header=T,blank.lines.skip = TRUE,na.strings = "NaN",row.names=1))
	rownames=row.names(ROImatrix)
		origcolnames = colnames(DesignMatrix);
	if (NroiC=="all") {
		rownames=row.names(ROImatrix)
		for (j in 1:length(rownames)) {
		roiName<-as.character(rownames[j])
#if (roiName !="Tract") {
				DesignMatrix[i,which(origcolnames==roiName)] = ROImatrix[which(rownames==roiName),1]
#}
		}
	} else if (Nroi > 0) {
		parsedROIs=parse(text=rois)
		for (nc in 1:Nroi) {
			roiName<-as.character(parsedROIs[nc])
			DesignMatrix[i,which(origcolnames==roiName)] = ROImatrix[which(rownames==roiName),1]
		}
	}
	}
}

iNA=which(apply(DesignMatrix,1,function(x)any(is.na(x))));
if (length(iNA > 0)) {
	DesignMatrix<-DesignMatrix[-which(apply(DesignMatrix,1,function(x)any(is.na(x)))),]
}

write.table(DesignMatrix,outTable,sep=",",row.names=F,col.names=T,quote=F);

In [ ]:
# Protocol for applying TBSS skeletonizations from FA analysis to diffusivity and obtaining ROI measures using the ENIGMA-DTI template
FSLDIR=/usr/local/fsl-5.0.7/

ENIGMAtemplateDirectory=/enigmaDTI/TBSS/ENIGMA_targets/
parentDirectory=/enigmaDTI/TBSS/run_tbss/
dtifit_folder=/enigmaDTI/DTIFIT/

mkdir ${parentDirectory}/MD/
mkdir ${parentDirectory}/AD/
mkdir ${parentDirectory}/RD/

cd $parentDirectory


for subj in subj_1 subj_2 … subj_N
do
   cp ${dtifit_folder}/${subj}*_MD.nii.gz ${parentDirectory}/MD/${subj}_MD.nii.gz
   cp ${dtifit_folder}/${subj}*_L1.nii.gz ${parentDirectory}/AD/${subj}_AD.nii.gz
   $FSLDIR/bin/fslmaths ${dtifit_folder}/${subj}*_L2.nii.gz –add ${dtifit_folder}/${subj}*_L3.nii.gz \\
       -div 2 ${parentDirectory}/RD/${subj}_RD.nii.gz


   for DIFF in MD AD RD
   do
   mkdir -p ${parentDirectory}/${DIFF}/origdata/
   mkdir -p ${parentDirectory}/${DIFF}_individ/${subj}/${DIFF}/
   mkdir -p ${parentDirectory}/${DIFF}_individ/${subj}/stats/

   $FSLDIR/bin/fslmaths ${parentDirectory}/${DIFF}/${subj}_${DIFF}.nii.gz -mas \\
      ${parentDirectory}/FA/${subj}_FA_FA_mask.nii.gz \\
      ${parentDirectory}/${DIFF}_individ/${subj}/${DIFF}/${subj}_${DIFF}

   $FSLDIR/bin/immv ${parentDirectory}/${DIFF}/${subj} ${parentDirectory}/${DIFF}/origdata/

   $FSLDIR/bin/applywarp -i ${parentDirectory}/${DIFF}_individ/${subj}/${DIFF}/${subj}_${DIFF} -o \\
      ${parentDirectory}/${DIFF}_individ/${subj}/${DIFF}/${subj}_${DIFF}_to_target -r \\
      $FSLDIR/data/standard/FMRIB58_FA_1mm -w ${parentDirectory}/FA/${subj}_FA_FA_to_target_warp.nii.gz

##remember to change ENIGMAtemplateDirectory if you re-masked the template

  $FSLDIR/bin/fslmaths ${parentDirectory}/${DIFF}_individ/${subj}/${DIFF}/${subj}_${DIFF}_to_target -mas \\
       ${ENIGMAtemplateDirectory}/ENIGMA_DTI_FA_mask.nii.gz \\
       ${parentDirectory}/${DIFF}_individ/${subj}/${DIFF}/${subj}_masked_${DIFF}.nii.gz

   $FSLDIR/bin/tbss_skeleton -i ./FA_individ/${subj}/FA/${subj}_masked_FA.nii.gz -p 0.049 \\
       ${ENIGMAtemplateDirectory}/ENIGMA_DTI_FA_skeleton_mask_dst.nii.gz $FSLDIR/data/standard/LowerCingulum_1mm.nii.gz \\         
       ${parentDirectory}/FA_individ/${subj}/FA/${subj}_masked_FA.nii.gz  \\
       ${parentDirectory}/${DIFF}_individ/${subj}/stats/${subj}_masked_${DIFF}skel -a \\
       ${parentDirectory}/${DIFF}_individ/${subj}/${DIFF}/${subj}_masked_${DIFF}.nii.gz -s \\
       ${ENIGMAtemplateDirectory}/ENIGMA_DTI_FA_skeleton_mask.nii.gz


   done
done


# Sanity Check
FSLDIR=/usr/local/fsl-5.0.7/

${FSLDIR}/bin/fslview  ${parentDirectory}/MD_individ/${subj}/stats/subj_1_masked_MDskel.nii.gz \\
     ${parentDirectory}/FA_individ/subj_1/stats/subj_1_masked_FAskel.nii.gz \\
     ${ENIGMAtemplateDirectory}/ENIGMA_DTI_FA_skeleton.nii.gz
 
# For all metrics
parentDirectory=/enigmaDTI/TBSS/run_tbss/
runDirectory=/enigmaDTI/TBSS/run_tbss/

for DIFF in MD AD RD
do
   mkdir ${parentDirectory}/${DIFF}_individ/${DIFF}_ENIGMA_ROI_part1
   dirO1=${parentDirectory}/${DIFF}_individ/${DIFF}_ENIGMA_ROI_part1/

   mkdir ${parentDirectory}/${DIFF}_individ/${DIFF}_ENIGMA_ROI_part2
   dirO2=${parentDirectory}/${DIFF}_individ/${DIFF}_ENIGMA_ROI_part2/

   for subject in subj_1 subj_2 … subj_N
     do

     ${runDirectory}/singleSubjROI_exe ${runDirectory}/ENIGMA_look_up_table.txt \\
         ${runDirectory}/mean_FA_skeleton.nii.gz ${runDirectory}/JHU-WhiteMatter-labels-1mm.nii.gz \\
         ${dirO1}/${subject}_${DIFF}_ROIout ${parentDirectory}/${DIFF}_individ/${subject}/stats/${subject}_masked_${DIFF}skel.nii.gz

     ${runDirectory}/averageSubjectTracts_exe ${dirO1}/${subject}_${DIFF}_ROIout.csv ${dirO2}/${subject}_${DIFF}_ROIout_avg.csv

# can create subject list here for part 3!
     echo ${subject},${dirO2}/${subject}_${DIFF}_ROIout_avg.csv >> ${parentDirectory}/${DIFF}_individ/subjectList_${DIFF}.csv
   done

   Table=${parentDirectory}/ROIextraction_info/ALL_Subject_Info.txt
   subjectIDcol=subjectID
   subjectList=${parentDirectory}/${DIFF}_individ/subjectList_${DIFF}.csv
   outTable=${parentDirectory}/${DIFF}_individ/combinedROItable_${DIFF}.csv
   Ncov=3  #2 if no disease
   covariates="Age;Sex;Diagnosis" # Just "Age;Sex" if no disease
   Nroi="all"
   rois="all"

#location of R binary
  Rbin=/usr/local/R-2.9.2_64bit/bin/R

#Run the R code
  ${Rbin} --no-save --slave --args ${Table} ${subjectIDcol} ${subjectList} ${outTable} \\
         ${Ncov} ${covariates} ${Nroi} ${rois} < ${runDirectory}/combine_subject_tables.R
done

In [ ]:
# Protocol for FA and Skeleton Visual QC analysis for ENIGMA-DTI
# MATLAB

TXTfile='/enigmaDTI/QC_ENIGMA/Subject_Path_Info.txt';
output_directory='/enigmaDTI/QC_ENIGMA/QC_FA_SKEL/';
[subjs,FAs,SKELs]=textread(TXTfile,'%s %s %s','headerlines',1)
for s = 1:length(subjs)
subj=subjs(s);
Fa=FAs(s);
skel=SKELs(s);
try
% reslice FA
[pathstrfa,nameniifa,gzfa] = fileparts(Fa{1,1});
[nafa,namefa,niifa] = fileparts(nameniifa);
newnamegzfa=[pathstrfa,'/',namefa,'_reslice.nii.gz'];
newnamefa=[pathstrfa,'/',namefa,'_reslice.nii'];
copyfile(Fa{1,1},newnamegzfa);
gunzip(newnamegzfa);
delete(newnamegzfa);
reslice_nii(newnamefa,newnamefa);
% reslice skel
[pathstrskel,nameniiskel,gzskel] = fileparts(skel{1,1});
[naskel,nameskel,niiskel] = fileparts(nameniiskel);
newnamegzskel =[pathstrskel,'/',nameskel,'_reslice.nii.gz'];
newnameskel =[pathstrskel,'/',nameskel,'_reslice.nii'];
copyfile(skel{1,1},newnamegzskel);
gunzip(newnamegzskel);
delete(newnamegzskel);
reslice_nii(newnameskel,newnameskel);
% qc
func_QC_enigmaDTI_FA_skel(subj,newnamefa,newnameskel,
output_directory);
close(1)
close(2)
close(3)
% delete
delete(newnamefa)
delete(newnameskel)
end
display(['Done with subject: ', num2str(s), ' of ',
num2str(length(subjs))]);
end

func_QC_enigmaDTI_FA_skel('subjectID', 'FA_image_path', 'Skel_image_path','output_directory')


# QC Webpage 
#!/bin/bash
#$ -S /bin/bash

###### no need to edit this script
###### to run >> ./make_enigmaDTI_FA_Skel_QC_webpage.sh ${dirO}  (where $dirO is the output directory specified for the Matlab script)

QC_directory=$1
cd $QC_directory
subject_list=`ls -d */`

echo "<html>" 												>  enigmaDTI_FA_Skel_QC.html
echo "<head>"                                                   >> enigmaDTI_FA_Skel_QC.html
echo "<style type=\"text/css\">"								>> enigmaDTI_FA_Skel_QC.html
echo "*"                                                        >> enigmaDTI_FA_Skel_QC.html
echo "{"														>> enigmaDTI_FA_Skel_QC.html
echo "margin: 0px;"												>> enigmaDTI_FA_Skel_QC.html
echo "padding: 0px;"											>> enigmaDTI_FA_Skel_QC.html
echo "}"														>> enigmaDTI_FA_Skel_QC.html
echo "html, body"												>> enigmaDTI_FA_Skel_QC.html
echo "{"														>> enigmaDTI_FA_Skel_QC.html
echo "height: 100%;"											>> enigmaDTI_FA_Skel_QC.html
echo "}"														>> enigmaDTI_FA_Skel_QC.html
echo "</style>"													>> enigmaDTI_FA_Skel_QC.html
echo "</head>"													>> enigmaDTI_FA_Skel_QC.html

echo "<body>" 													>>  enigmaDTI_FA_Skel_QC.html


for sub in ${subject_list};
do
echo "<table cellspacing=\"1\" style=\"width:100%;background-color:#000;\">"				>> enigmaDTI_FA_Skel_QC.html
echo "<tr>"																					>> enigmaDTI_FA_Skel_QC.html
echo "<td> <FONT COLOR=WHITE FACE=\"Geneva, Arial\" SIZE=5> $sub </FONT> </td>"             >> enigmaDTI_FA_Skel_QC.html
echo "</tr>"                                                                                >> enigmaDTI_FA_Skel_QC.html
echo "<tr>"                                                                                 >> enigmaDTI_FA_Skel_QC.html
echo "<td><a href=\"file:"$sub"/Axial_Skel_check.png\"><img src=\""$sub"/Axial_Skel_check_lowRez.png\" width=\"100%\" ></a></td>"	>> enigmaDTI_FA_Skel_QC.html
echo "<td><a href=\"file:"$sub"/Coronal_Skel_check.png\"><img src=\""$sub"/Coronal_Skel_check_lowRez.png\" width=\"100%\" ></a></td>"	>> enigmaDTI_FA_Skel_QC.html
echo "<td><a href=\"file:"$sub"/Sagittal_Skel_check.png\"><img src=\""$sub"/Sagittal_Skel_check_lowRez.png\" width=\"100%\" ></a></td>"	>> enigmaDTI_FA_Skel_QC.html
echo "</tr>"																				>> enigmaDTI_FA_Skel_QC.html
echo "</table>"                                                                             >> enigmaDTI_FA_Skel_QC.html
done;

echo "</body>"                                                                              >> enigmaDTI_FA_Skel_QC.html
echo "</html>"   >> enigmaDTI_FA_Skel_QC.html


# Make it executable 
chmod 777 make_enigmaDTI_FA_Skel_QC_webpage.sh

./make_enigmaDTI_FA_Skel_QC_webpage.sh /enigmaDTI/QC_ENIGMA/QC_FA_SKEL/

firefox /enigmaDTI/QC_ENIGMA/QC_FA_SKEL/enigmaDTI_FA_Skel_QC.html


In [ ]:
# Checking mean and max projection distances
#!/bin/sh
# Emma Sprooten for ENIGMA-DTI
# run in a new directory eg. Proj_Dist/
# create a text file containing paths to your masked FA maps
# output in Proj_Dist.txt

# make sure you have FSL5!!!

###### USER INPUTS ###############
## insert main folder where you ran TBSS
## just above "stats/" and "FA/"
maindir="/enigmaDTI/TBSS/run_tbss/"
list=`find $maindir -wholename "*/FA/*_masked_FA.nii.gz"`

## insert full path to mean_FA, skeleton mask and distance map
## based on ENIGMA-DTI protocol this should be:
mean_FA="/enigmaDTI/TBSS/ENIGMA_targets/mean_FA_MyMasked.nii.gz"
mask="/enigmaDTI/TBSS/ENIGMA_targets/mean_FA_skeleton_MyMasked.nii.gz"
dst_map="/enigmaDTI/TBSS/ENIGMA_targets/enigma_skeleton_mask_dst.nii.gz"

##############
### from here it should be working without further adjustments

rm Proj_Dist.txt
echo "ID" "Mean_Squared" "Max_Squared" >> Proj_Dist.txt


## for each FA map
    for FAmap in ${list}   
    do
	base=`echo $FAmap | awk 'BEGIN {FS="/"}; {print $NF}' | awk 'BEGIN {FS="_"}; {print $1}'`
        dst_out="dst_vals_"$base""

	# get Proj Dist images
        tbss_skeleton -d -i $mean_FA -p 0.2 $dst_map $FSLDIR/data/standard/LowerCingulum_1mm $FAmap $dst_out

	#X direction
	Xout=""squared_X_"$base"
	file=""$dst_out"_search_X.nii.gz"
	fslmaths $file -mul $file $Xout

	#Y direction
	Yout=""squared_Y_"$base"
	file=""$dst_out"_search_Y.nii.gz"
	fslmaths $file -mul $file $Yout

	#Z direction
        Zout=""squared_Z_"$base"
        file=""$dst_out"_search_Z.nii.gz"
	fslmaths $file -mul $file $Zout

	#Overall displacement
	Tout="Total_ProjDist_"$base""
	fslmaths $Xout -add $Yout -add $Zout $Tout

	# store extracted distances
	mean=`fslstats -t $Tout -k $mask -m`  
	max=`fslstats -t $Tout -R | awk '{print $2}'`
        echo "$base $mean $max" >> Proj_Dist.txt

        # remove X Y Z images
        ## comment out for debugging
        rm ./dst_vals_*.nii.gz
        rm ./squared_*.nii.gz

	echo "file $Tout done"
    done

In [ ]:
# Protocol for Creating Histograms and Summary Stats
mkdir /enigmaDTI/figures/

cp /enigmaDTI/combinedROItable.csv /enigmaDTI/figures/

mv /enigmaDTI/downloads/ENIGMA_DTI_plots.R /enigmaDTI/figures/

cd /enigmaDTI/figures

# New directory for histogram plots = /enigmaDTI/figures/QC_ENIGMA

cohort= 'MyCohort'
R --no-save --slave --args ${cohort} < ENIGMA_DTI_plots_ALL.R

# R script 
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#                        %%%  ENIGMA DTI %%%
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%% This is a function to print out images for Quality Control
#%% of DTI_ENIGMA FA images with TBSS (FSL) skeltons overlaid
#%% as well as JHU atlas ROIs
#%%
#%% Please QC your images to make sure they are
#%% correct FA maps and oriented and aligned properly
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%% Writen by Neda Jahanshad / Derrek Hibar
#%%   last update June 2014
#%%           Questions or Comments??
#%% neda.jahanshad@ini.usc.edu / derrek.hibar@ini.usc.edu
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

cmdargs = commandArgs(trailingOnly=T);
site=cmdargs[1];  

outD = '.QC_ENIGMA/'
CSVfile ='combinedROItable.csv'
Nrois = 63;
rois = "AverageFA;BCC;GCC;SCC;CC;CGC;CGH;CR;EC;FX;FXST;IC;IFO;PTR;SFO;SLF;SS;UNC;CST;ACR;ALIC;PCR;PLIC;RLIC;SCR;ACR.L;ACR.R;ALIC.L;ALIC.R;CGC.L;CGC.R;CGH.L;CGH.R;CR.L;CR.R;CST.L;CST.R;EC.L;EC.R;FX.ST.L;FX.ST.R;IC.L;IC.R;IFO.L;IFO.R;PCR.L;PCR.R;PLIC.L;PLIC.R;PTR.L;PTR.R;RLIC.L;RLIC.R;SCR.L;SCR.R;SFO.L;SFO.R;SLF.L;SLF.R;SS.L;SS.R;UNC.L;UNC.R"
outPDF = 'ENIGMA_DTI_allROI_histograms.pdf'
outTXT = 'ENIGMA_DTI_allROI_stats.txt'

###########################################################################

dir.create(outD)

Table <- read.csv(CSVfile,header=T);
colTable = names(Table);
print(colTable)
## assigning all rows that have a value of "x" or "X" to "NA"
for (m in seq(1,length(colTable)))
{
	ind = which(Table[,m]=="x");
	ind2 = which(Table[,m]=="X");
	Table[ind] = "NA"
	Table[ind2] = "NA"
}

##get rid of all rows with NAs in them
INDX=which(apply(Table,1,function(x)any(is.na(x))));

##get rid of all rows with NAs in them
if (length(INDX) >0 )
{
	Table<-Table[-which(apply(Table,1,function(x)any(is.na(x)))),]
}

## parsing through the inputted list of ROIs
if (Nrois > 0)
{
	pdf(file=outPDF);
	
	parsedROIs = parse(text=rois);
    print(parsedROIs)
    write("Structure\tNumberIncluded\tMean\tStandDev\tMaxValue\tMinValue\tMaxSubject\tMinSubject\t5StDev_Off", file = outTXT);
    
	for (x in seq(1,Nrois,1))
	{
		
	ROI <- as.character(parsedROIs[x]);
	
	DATA = Table[ROI];
	DATA = unlist(DATA);
	DATA = as.numeric(as.vector(DATA));
	
	mu = mean(DATA);
	sdev = sd(DATA);
	N = length(DATA);
    
    outliers<-"";
	
    hbins = 20; #floor(N/10);
	
	maxV = max(DATA);
	minV = min(DATA);
	
	i =which(DATA==maxV)
	maxSubj = as.character(Table[i,1])
	
	j =which(DATA==minV)
	minSubj = as.character(Table[j,1])
    
    minO=which(DATA < mu - 5*sdev)
    maxO=which(DATA > mu + 5*sdev)
    
    if (length(minO) + length(maxO) > 0 ) {
        outliers<-"Outliers (5-sd):";
        for (ii in 1:length(minO)){
        outliers=paste(outliers,as.character(Table[minO[ii],1]),sep=",")
        }
        for (ii in 1:length(maxO)){
        outliers=paste(outliers,as.character(Table[maxO[ii],1]),sep=",")
        }}
    
    outliers=as.character(outliers)
	
	stats = c(ROI, N, mu, sdev, maxV, minV, maxSubj, minSubj,outliers);
	
	
	write.table(t(as.matrix(stats)),file = outTXT, append=T, quote=F, col.names=F,row.names=F, sep="\t");
		
	hist(DATA, breaks = hbins, main = paste(site,ROI));
	
	## uncomment the following 3 lines if you want to output individual histogram PNGs for each inputted ROI
        # png(paste(outD,ROI,"hist_data.png"));
        # hist(DATA, breaks = hbins, main = ROI);
        # dev.off()
	}
dev.off()
}
